<a href="https://colab.research.google.com/github/paridhika/DDL/blob/main/ML_split_Cost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# According to PyTorch

In [ ]:
from torchvision.models import resnet50
import torch
from torchsummary import summary

In [ ]:
summary(resnet50(), (3, 224, 224))

# According to TF

In [ ]:
import tensorflow as tf

In [41]:
# model = tf.keras.applications.resnet.ResNet50(weights=None)
model =  tf.keras.applications.InceptionResNetV2(weights=None) 
model.summary();

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_406 (Conv2D)            (None, 149, 149, 32  864         ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_406 (Batch  (None, 149, 149, 32  96         ['conv2d_406[0][0]']             
 Normalization)                 )                                               

# Get layers with tensors

In [ ]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
my_map = {}
for i, layer in enumerate(model.layers):
    if "add" not in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))

map_list = list(my_map.items())
map_list.reverse()
with open("tensor_layers.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)

161


# Get layers with parameters

In [ ]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
my_map = {}
for i, layer in enumerate(model.layers):
    if len(layer.weights) > 0:
      if "add" not in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))

map_list = list(my_map.items())
map_list.reverse()
with open("parameters_layers.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)

107


# Compute burst size

In [ ]:
from tensorflow.python.ops.math_ops import sub
# import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model.summary();
import csv  
layer_count = 0
substring = ""
my_map = {}


for i, layer in enumerate(model.layers):
    if len(layer.weights) > 0:
      if "_" in layer.name:
        layer_count += 1
        # print("Layer {}: {} ({} parameters)".format(layer_count, layer.name, layer.count_params()))
        substring = layer.name[:layer.name.rindex("_")]
        if substring in my_map:
          my_map[substring] = my_map[substring] + layer.count_params()
        else:
          my_map[substring] = layer.count_params()
      else:
        my_map[layer.name] = layer.count_params()
  
print(len(my_map))
my_map["Layer Name"] = "Theoritical"
map_list = list(my_map.items())
map_list.reverse()
with open("my_map.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(map_list)





54


**Get table of comparison between theoritical and experimental burst sizes**

In [ ]:
import csv
import pandas as pd
def generate_latex_table(filename):
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        header = next(reader)
        table = []
        for row in reader:
            table.append(row)
        
    table_string = "\\begin{tabular}{|" + "c|"*len(header) + "}\n\\hline\n"
    table_string += " & ".join(header) + "\\\\\n\\hline\n"
    for row in table:
        table_string += " & ".join(row) + "\\\\\n\\hline\n"
    table_string += "\\end{tabular}"
    
    return table_string

numbers = [2049839.5,1048383.5,2359135.5,1048447.5,1048367.5,2359167.5,1048367.5,1048367.5,2096959.5,2359311.5,524079.5,262015.5,589647.5,261983.5,261951.5,589695.5,261951.5,262015.5,589615.5,261935.5,261935.5,589663.5,262015.5,262527.5,589615.5,261919.5,261935.5,524143.5,589599.5,130943.5,65359.5,147231.5,65375.5,65551.5,147247.5,65343.5,65391.5,147231.5,65327.5,65551.5,130879.5,147231.5,0,0,0,0,0,0,0,0,0,0,0,0]

rounded_values = [int(round(number)) for number in numbers]

# Load the existing CSV file into a pandas dataframe
df = pd.read_csv("my_map.csv")

# Create a new column in the dataframe with the rounded integer values
df["Experimental"] = rounded_values

# Save the updated dataframe to the existing CSV file
df.to_csv("my_map.csv", index=False)

filename = 'my_map.csv'
index_col = [i for i in range(1, 55)] #index column
index_col = index_col[::-1]
with open(filename, 'r') as file:
    reader = csv.reader(file)
    header = next(reader) #get header
    header.insert(0, 'Burst Count') #insert "Index" header in the first position
    data = [row for row in reader] #get data

#create new data with index column
new_data = []
for i, row in enumerate(data):
    row.insert(0, index_col[i])
    new_data.append(row)

#write data to a new file
with open('new_file.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(new_data)



In [ ]:
import csv

# Open the CSV file for reading
with open('new_file.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    data = [row for row in reader]

# Round off all floating point values to integers
for i, row in enumerate(data):
    data[i] = [int(float(cell)) if '.' in cell else cell for cell in row]

# Write the updated data back to the same CSV file
with open('final.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)



In [ ]:
table_string = generate_latex_table("new_file.csv")
with open('latex_bursts.tex', 'w') as f:
        f.write(table_string)

**Compute number of parameters by formula**

In [ ]:
import tensorflow as tf

# Instantiate the ResNet50 model
model =  tf.keras.applications.resnet.ResNet50(weights=None) 

# Define a dictionary to store the number of operations for each layer
ops_dict = {}
param_dict = {}
# Define a function to compute the number of operations for a Conv2D layer
def compute_ops_conv2d(layer):
    kernel_size = layer.kernel_size[0]
    in_channels = layer.input_shape[-1]
    out_channels = layer.output_shape[-1]
    ops = kernel_size**2 * in_channels * out_channels
    return ops

# Define a function to compute the number of operations for a MaxPooling2D layer
def compute_ops_maxpool2d(layer):
    pool_size = layer.pool_size[0]
    in_channels = layer.input_shape[-1]
    ops = pool_size**2 * in_channels
    return ops

# Define a function to compute the number of operations for a ReLU layer
def compute_ops_relu(layer):
    num_elements = tf.reduce_prod(layer.input_shape[1:])
    ops = num_elements
    return ops

# Define a function to compute the number of operations for the output layer
def compute_ops_output(layer):
    num_input_channels = layer.input_shape[-1]
    num_output_channels = layer.output_shape[-1]
    fc_ops = (num_input_channels * num_output_channels) + num_output_channels
    if type(layer.activation) == tf.keras.activations.softmax or type(layer.activation) ==  tf.keras.activations.sigmoid:
        num_elements = tf.reduce_prod(layer.output_shape[1:])
        act_ops = num_elements
    else:
        act_ops = 0
    ops = fc_ops + act_ops
    return ops

# Compute the number of operations for each layer in the model
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        ops = compute_ops_conv2d(layer)
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        ops = compute_ops_maxpool2d(layer)
    elif isinstance(layer, tf.keras.layers.Activation) and type(layer.activation) == tf.keras.activations.relu:
        ops = compute_ops_relu(layer)
    elif isinstance(layer, tf.keras.layers.Dense):
        ops = compute_ops_output(layer)
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        num_channels = layer.output_shape[-1]
        ops = 4 * num_channels
    else:
        ops = 0
    ops_dict[layer.name] = ops
    param_dict[layer.name] = layer.count_params()
# Print the number of operations for each layer in the model
for layer_name, ops in ops_dict.items():
    print(layer_name, ops, param_dict[layer_name])


**Make plot for comparison**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the csv file into a pandas DataFrame
df = pd.read_csv('my_map.csv')

# Extract the two columns you need
col1 = df['Theoritical']
col2 = df['Experimental']

# Plot the histogram
plt.plot(df.iloc[:43, 1]/1000000, color='blue', label='Theoritical')
plt.plot(df.iloc[:43, 2]/1000000, color='red', label='Experimental')

# Add a legend and labels for the x and y axes
plt.legend()
plt.xlabel('Count')
plt.ylabel('Data (MB)')

plt.show()

# Compute computation time

In [ ]:
import tensorflow as tf

# Instantiate the ResNet50 model
model =  tf.keras.applications.resnet.ResNet50(weights=None) 

# Print the input and output shape for each convolutional layer in the model
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
      print(layer.name, layer.input_shape, layer.output_shape,  layer.kernel_size, layer.filters)

print(len(model.layers))


In [30]:
#  Edge : Model	 i9-11900K;
#         Cores/Threads	8/16
#         Base Clock	3.5 GHz
#         Boost Clock	5.3 GHz
#         Cache	  16 MB
#         TDP	  125W
#         GFLOPs  925.6

# Raspberry Pi 4 
# GFLOPs 24 
# Memory 1 to 8 GB


Edge = 925
CE = 0

# Near the Edge : Model Intel Xeon E5-2666 v3	
# c4.16xlarge	vCPUs 64	GFLOPs 2144.0 	$1.75/
# c4.8xlarge	vCPUs 36	Memory 60 GiB	EBS only	On Demand $1.59/hr	Reserved $1.01/hr	
# c4.large	vCPUs 2	Memory 3.75 GiB	EBS only	On Demand $0.10/hr	Reserved $0.06/hr	GFLOPs 67.2

# c7g

NTE = 3500
CNTE = 1.59/3600 # cost per second

# Cloud : inf1.24xlarge	8x AWS Inferentia chips with 16 Neuron Cores each	
#  Memory 192 (GiB)	TFLOPs	42.0
# inf1.24xlarge	vCPUs 96	Network b/w	100	EBS b/w 19	$4.721/Hr
# inf1.xlarge	vCPUs 4	Memory 8 GiB	EBS only	On Demand $0.23/hr	Reserved $0.14/hr GFLOPs = 2250
# inf2.48xlarge	vCPUs 192	Memory 768 GiB	EBS only	On Demand $12.98/hr	Reserved $8.18/hr GFLOPs = 8,400,000

# T4 GPU 65 TFLOPs
Cloud = 65000
# CC = 2.74/3600  # cost per second
CC = 4.352/3600  # cost per second


# cost model from Bell Network charges $0.3078 for transferring 1GB over its OC3 link -> 150 Mbps
TCost = 307.8 # per MB
# print(f'Cost for transmission : ${params/TCost}')

In [58]:
from keras.api._v2.keras.layers import Activation
import tensorflow as tf
import numpy as np


# Instantiate the ResNet50 model
# model =  tf.keras.applications.resnet.ResNet50(weights=None) 
# model = tf.keras.applications.vgg16.VGG16(weights=None) 
# model =  tf.keras.applications.resnet.ResNet152(weights='imagenet') 
# model =  tf.keras.applications.InceptionResNetV2(weights=None) 
# model.summary()

# # Define the input shape of the model
# input_shape = model.input_shape[1:]

# # Generate a random image with the input shape of the model
# random_image = np.random.rand(*input_shape)

# # Preprocess the image
# preprocessed_image = tf.keras.applications.resnet50.preprocess_input(random_image)

# # Make a prediction with the model
# prediction = model.predict(np.array([preprocessed_image]))

# # Decode the prediction
# decoded_prediction = tf.keras.applications.resnet50.decode_predictions(prediction, top=1)[0][0]

# # Print the decoded prediction
# print(decoded_prediction)

# Define a dictionary to store the number of operations for each layer
ops_dict = {}

batch_size = 1
# model_input_shape = (batch_size, 224, 224, 3)

# Define a function to compute the number of operations for a Conv2D layer
def compute_ops_conv2d(layer):
    kernel_size = layer.kernel_size[0]
    in_channels = layer.input_shape[-1]
    out_channels = layer.output_shape[-1]
    input_height, input_width = layer.input_shape[1:3]
    output_height, output_width = layer.output_shape[1:3]
    ops = kernel_size**2 * in_channels * output_height * output_width * out_channels
    return ops * batch_size

# Define a function to compute the number of operations for a MaxPooling2D layer
def compute_ops_maxpool2d(layer):
    pool_size = layer.pool_size[0]
    in_channels = layer.input_shape[-1]
    input_height, input_width = layer.input_shape[1:3]
    output_height, output_width = layer.output_shape[1:3]
    ops = pool_size**2 * in_channels * output_height * output_width
    return ops * batch_size

# Define a function to compute the number of operations for a BatchNormalization layer
def compute_ops_batchnorm(layer):
    # Get input shape
    input_shape = layer.input_shape[1:]
    in_channels = input_shape[-1]

    # Number of operations per element (2 for mean and variance)
    ops = 2 * in_channels

    return ops * batch_size

# Define a function to compute the number of operations for a ReLU layer
def compute_ops_relu(layer):

    input_shape = layer.input_shape[1:]
    # Number of operations per element
    ops = 1
    for dim in input_shape:
        ops *= dim

    return ops * batch_size

# Define a function to compute the number of operations for the output layer
def compute_ops_output(layer):
    num_input_channels = layer.input_shape[-1]
    num_output_channels = layer.output_shape[-1]
    fc_ops = (num_input_channels * num_output_channels) + num_output_channels
    if type(layer.activation) == tf.keras.activations.softmax or type(layer.activation) == tf.keras.activations.sigmoid:
        num_elements = tf.reduce_prod(layer.output_shape[1:])
        act_ops = num_elements
    else:
        act_ops = 0
    ops = fc_ops + act_ops
    return ops * batch_size

# Print the number of operations for each layer in the model
sum = 0;


# Compute the number of operations for each layer in the model
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        ops = compute_ops_conv2d(layer)
    elif isinstance(layer, tf.keras.layers.MaxPooling2D):
        ops = compute_ops_maxpool2d(layer)
    elif isinstance(layer, tf.keras.layers.BatchNormalization):
        ops = compute_ops_batchnorm(layer)
    elif isinstance(layer, tf.keras.layers.Activation):
        ops = compute_ops_relu(layer)
    elif isinstance(layer, tf.keras.layers.Dense):
        ops = compute_ops_output(layer)
    else:
        ops = 0
    ops_dict[layer.name] = [ops, layer.input_shape, layer.output_shape]
    # print(layer.name, ops, layer.input_shape)

i = 0
layer_map = {}
with open("layer_list.csv", "w", newline="") as f:
  writer = csv.writer(f)
  writer.writerow(["Layer Name","Complexity (GFLOP)","Input Size (MB)"])
  layer_map["Layer Name"] = ["Complexity (GFLOP)","Input Size (MB)"]
  for layer_name, (ops, inp, out) in ops_dict.items():
      sum = sum + ops
      inp_size = 0

      if inp[0] is not None:
        (n,l,h,w) = inp[0]
        inp_size += l*h*w*batch_size/1000000
        
        if len(inp) == 2:
          (n,l,h,w) = inp[1]
          inp_size += l*h*w*batch_size/1000000

      elif len(inp) > 2:
        inp_size += inp[1]*inp[2] * inp[3] * batch_size/1000000
      else:
        inp_size += inp[1] * batch_size/1000000

      if i == 0:
        (n,l,h,w) = inp[0]
        i = i+10
        params = l*h*w
      # print(layer_name, ops, inp_size)
      layer_map[layer_name] = [ops/1000000000,inp_size]
      if ops != 0:
        writer.writerow([layer_name,ops/1000000000,inp_size])

sum = sum/1000000000 #GFLOP
GFLOPs = sum
param = params/1000000
print(f'input image : {params}')
print(f'GFLOP : {GFLOPs}')
arr_GFLOP = []
arr_input = []
total_cost = []
total_time = []
for batch_size in [1,32,64,128]:
  params = param * batch_size 
  GFLOP = GFLOPs * batch_size
  # print(f'GFLOP : {GFLOP * batch_size}')
  # print(f'Total input images : {params * batch_size} MB')
  # print()

  arr_GFLOP.append(GFLOP)
  arr_input.append(params)

  # Bandwidth = 20 mbps 
  # Transmission time from E to NTE
  BWENTE = 20
  TENTE = params/BWENTE

  PENTE = 0.001 # 1 ms
  PNTEC = 0.1 # 100 ms
  # print(f'Transmission Time from Edge to Near the Edge : {TENTE} sec')

  # print(f'Propagation Delay from Edge to Near the Edge : {PENTE} sec')



  # Bandwidth = 20 Mbps
  # Transmission time from E to Cloud
  BWNTEC = 20
  TNTEC = params/BWNTEC

  # print(f'Transmission Time from Near the Edge to Cloud : {TNTEC} sec')
  # print(f'Propagation Delay from Near the Edge to Cloud: {PNTEC} sec')

  TEC = TENTE + TNTEC
  PEC = PENTE + PNTEC
  # print(f'Transmission Time from Edge to Cloud : {TEC} sec')
  # print(f'Propagation Delay from Edge to Cloud: {PEC} sec')

  TE = GFLOP / Edge
  TNTE = GFLOP / NTE
  TC = GFLOP / Cloud

  # print(f'Time for excecution at Edge : {TE} secs')
  # print(f'Time for excecution at NTE : {TNTE} secs')
  # print(f'Time for excecution at Cloud : {TC} secs')

  # print()

  # print(f'Transmission Time from Edge to Near the Edge : {TENTE} sec')
  # print(f'Transmission Time from Edge to Cloud : {TEC} sec')
  # print()

  # print(f'Total time at Near the Edge : {TNTE + TENTE + PENTE} sec')
  # print(f'Total time at Cloud : {TEC + TC + PEC} sec')

  time = []
  time.append(TE)
  time.append(TNTE + TENTE + PENTE)
  time.append(TEC + TC + PEC)
  total_time.append(time)
  cost = []
  MCE = TE * CE
  MCNTE = TNTE * CNTE 
  MCC = TC * CC
  cost.append(MCE)
  cost.append(MCNTE)
  cost.append(MCC)
  total_cost.append(cost)
  # print()
  # print(f'Cost for excecution at Edge : ${MCE}')
  # print(f'Cost for excecution at NTE : ${MCNTE}')
  # print(f'Cost for excecution at Cloud : ${MCC}')
  # print(f'Parameters Memory : 102 MB')


input image : 150528
GFLOP : 3.86891556


In [40]:
import csv  
with open("store_costs.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Batch Size","Computation complexity(GFLOP)", "Input Size (MB)", "Total time at Edge", "Total time at Near the Edge", "Total time at Cloud",  "Total cost at Edge", "Total cost at Near the Edge", "Total cost at Cloud"])
    i=0
    for batch_size in [1,32,64,128]:
      writer.writerow([batch_size,arr_GFLOP[i],arr_input[i],total_time[i][0],total_time[i][1],total_time[i][2],total_cost[i][0],total_cost[i][1], total_cost[i][2]])
      i += 1
# print(total_time[0][3])

In [89]:
# Bandwidth = 1 Gbps 
# Transmission time from E to NTE
BWENTE = 20
TENTE = params/BWENTE

PENTE = 0.001 # 1 ms
PNTEC = 0.1 # 100 ms
print(f'Transmission Time from Edge to Near the Edge : {TENTE} sec')

print(f'Propagation Delay from Edge to Near the Edge : {PENTE} sec')



# Bandwidth = 20 Mbps
# Transmission time from E to Cloud
BWNTEC = 20
TNTEC = params/BWNTEC

print(f'Transmission Time from Near the Edge to Cloud : {TNTEC} sec')
print(f'Propagation Delay from Near the Edge to Cloud: {PNTEC} sec')

TEC = TENTE + TNTEC
PEC = PENTE + PNTEC
print(f'Transmission Time from Edge to Cloud : {TEC} sec')
print(f'Propagation Delay from Edge to Cloud: {PEC} sec')

TE = GFLOP / Edge
TNTE = GFLOP / NTE
TC = GFLOP / Cloud

print(f'Time for excecution at Edge : {TE} secs')
print(f'Time for excecution at NTE : {TNTE} secs')
print(f'Time for excecution at Cloud : {TC} secs')

print()

print(f'Transmission Time from Edge to Near the Edge : {TENTE} sec')
print(f'Transmission Time from Edge to Cloud : {TEC} sec')
print()

print(f'Total time at Near the Edge : {TNTE + TENTE + PENTE} sec')
print(f'Total time at Cloud : {TEC + TC + PEC} sec')

MCE = TE * CE
MCNTE = TNTE * CNTE 
MCC = TC * CC
print()
print(f'Cost for excecution at Edge : ${MCE}')
print(f'Cost for excecution at NTE : ${MCNTE}')
print(f'Cost for excecution at Cloud : ${MCC}')

Transmission Time from Edge to Near the Edge : 0.9633792 sec
Propagation Delay from Edge to Near the Edge : 0.001 sec
Transmission Time from Near the Edge to Cloud : 0.9633792 sec
Propagation Delay from Near the Edge to Cloud: 0.1 sec
Transmission Time from Edge to Cloud : 1.9267584 sec
Propagation Delay from Edge to Cloud: 0.101 sec
Time for excecution at Edge : 0.5353742612756757 secs
Time for excecution at NTE : 0.386891556 secs
Time for excecution at Cloud : 0.007618787564307693 secs

Transmission Time from Edge to Near the Edge : 0.9633792 sec
Transmission Time from Edge to Cloud : 1.9267584 sec

Total time at Near the Edge : 1.351270756 sec
Total time at Cloud : 2.0353771875643076 sec

Cost for excecution at Edge : $0.0
Cost for excecution at NTE : $0.00010854457543333333
Cost for excecution at Cloud : $5.798743868389744e-06


In [ ]:
#  Memory required by ResNet152 = 242 MB (parameter) + 200 KB ( for intermediate activations)
Mem = 242 # MB

# GPT2 Model

In [4]:
# !pip install transformers

from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
# print(len(model.layers))

In [ ]:
for name, param in model.named_parameters():
    if 'weight' in name:
        print(f'Layer: {name}, Shape: {param.shape}')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Model

from torch.profiler import profile, record_function, ProfilerActivity

# Define the GPT-2 model
model = GPT2Model.from_pretrained('gpt2')

# Define the input
input_ids = torch.randint(0, 1000, (1, 128))

# Define the profiler
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    # Run the model
    with record_function("model_inference"):
        with torch.no_grad():
            output = model(input_ids)

# Print the profiler results
print(prof.key_averages(group_by_input_shape=True).table(sort_by="self_cpu_time_total", row_limit=10))


In [ ]:
import torch
from torchprofile import profile_macs
from torchsummary import summary
from transformers import GPT2Tokenizer, GPT2Model

# Load the model
# model = GPT2Model.from_pretrained('gpt2')

# Create a sample input
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
input_str = 'Hello, how are you today?'
input_ids = torch.tensor([tokenizer.encode(input_str)])

# Compute the FLOPs of each layer in the model
macs_list = [profile_macs(layer, (input_ids,)) for layer in model.modules()]
for i, macs in enumerate(macs_list):
    if macs is not None:
        print(f"Layer {i + 1} FLOPs: {macs}")



In [ ]:
import torch
from transformers import GPT2Model
from torchprofile import profile_macs

# define the model and input
# model = GPT2Model.from_pretrained('gpt2')
# input_ids = torch.randint(low=0, high=1000, size=(1, 512))

# compute the FLOPs of each layer in the model
macs_list = {}
for module in model.modules():
    if hasattr(module, 'forward') and module.forward.__code__.co_argcount > 1:
        macs_list.append(profile_macs(module, (input_ids,)))


for i, macs in enumerate(macs_list):
    print(f"Layer {i + 1} FLOPs: {macs}")


In [64]:
# !pip install thop
# !pip nstall transformers
import torch
from transformers import GPT2Model
from thop import profile
from torchsummary import summary

# define the model and input
# model = GPT2Model.from_pretrained('gpt2')
input_ids = torch.randint(low=0, high=1000, size=(1, 512))
# summary(model, [(1, 512)])
# compute the FLOPs of each layer in the model
flops_list = []
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Linear) or isinstance(module, torch.nn.MultiheadAttention):
        input_size = (input_ids.size(0),) + tuple(module.in_proj_weight.shape[1:])
        flops, params = profile(module, inputs=(input_ids,), verbose=False)
        flops_list.append((name, flops))
        
for i, (name, flops) in enumerate(flops_list):
    print(f"Layer {i + 1} ({name}) FLOPs: {flops}")




RuntimeError: ignored

In [17]:
!pip install tensorboard


In [24]:
from tensorflow.keras import plot_model
from datetime import datetime

# Set up log directory for TensorBoard
logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")

# Define a Keras callback for TensorBoard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# Plot the model graph and save as image file
plot_model(model, to_file='resnet50.png')

# Launch TensorBoard
!tensorboard --logdir logs


2023-05-05 17:00:41.145573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.33' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
/usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /usr/local/lib/python3.10/dist-packages/tensorboard_data_server/bin/server)
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.2 at http://localhost:6006/ (Press CTRL+C to quit)


# Understanding Model Summary

In [2]:
import tensorflow as tf
# model = tf.keras.applications.resnet.ResNet50(weights='imagenet');
# model =  tf.keras.applications.InceptionResNetV2(weights=None) 
# model.summary();
print(len(model.layers))
trainableLayerNameList = [layer.name for layer in model.layers if layer.trainable_variables]
# print(len(trainableLayerNameList));

782


In [ ]:
trainable = 0

# iterate over the layers in the model
for layer in model.layers:
    # check if the layer has trainable parameters
    if layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        trainable += 1

# print the number of layers involved in the AllReduce operation
print(f'Number of layers with trainable parameters: {trainable}')

Number of layers with trainable parameters: 107


In [ ]:
allreduce_layers = 0;
for layer in model.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        # if the layer has trainable parameters, increment the counter
        if "add" in layer.name:
          allreduce_layers += 1
          
# print the number of layers involved in the AllReduce operation
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')
print(f'Number of layers involved in AllReduce: {len(model.layers) - allreduce_layers}')

In [ ]:
# import tensorflow as tf
resnet101 = tf.keras.applications.resnet.ResNet101(weights='imagenet')
len(resnet101.layers)
trainableLayerNameList = [layer.name for layer in resnet101.layers if layer.trainable_variables]
print(len(trainableLayerNameList));
allreduce_layers = 0;
for layer in resnet101.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        if "add" in layer.name:
          allreduce_layers += 1
          # print(layer.name);
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

In [ ]:
vgg16 = tf.keras.applications.vgg16.VGG16(weights=None)
vgg16.summary();
print(f'Number of layers : {len(vgg16.layers)}')
trainableLayerNameList = [layer.name for layer in vgg16.layers if layer.trainable_variables]
# print(len(trainableLayerNameList));
allreduce_layers = 0;
for layer in vgg16.layers:
    # check if the layer has trainable parameters
    if not layer.trainable_variables:
        if "add" in layer.name:
          allreduce_layers += 1
          # print(layer.name);
print(f'Number of layers not involved in AllReduce: {allreduce_layers}')

In [ ]:
import re
conv2d = 0;
for layer in vgg16.layers:
  if "_conv" in layer.name:
    conv2d += 1;
print(conv2d);

13


In [ ]:
print(trainableLayerNameList)

# DNN Computational Graph Visualization

In [ ]:
# !pip install torchviz
import torch
import torchviz
import torchvision.models as models

# Load the ResNet50 model
resnet = models.resnet50(pretrained=False)

# Create a random input tensor
x = torch.randn(1, 3, 224, 224)

# Generate the output tensor and the computational graph
y = resnet(x)
dot = torchviz.make_dot(y, params=dict(resnet.named_parameters()))

# Save the graph to a PDF file
dot.render('resnet50_graph')

# Display the graph in Jupyter notebook (optional)
# dot


In [ ]:
import torch
import torch.nn as nn
import torchviz

# Load the ResNet50 model
resnet = torch.hub.load('pytorch/vision', 'resnet50', pretrained=False)

# Create a random input tensor
x = torch.randn(1, 3, 224, 224)

# Generate the output tensor and the computational graph
y = resnet(x)
graph = torchviz.make_dot(y, params=dict(resnet.named_parameters()))

# Get the input size for each layer
input_sizes = {}
for name, module in resnet.named_modules():
    if isinstance(module, nn.Conv2d):
        input_sizes[name] = x.shape[2:]
    elif isinstance(module, nn.MaxPool2d):
        x = module(x)
        input_sizes[name] = x.shape[2:]

# Print the input size for each layer
for name, size in input_sizes.items():
    print(f"{name}: {size}")

# Save the graph in a PDF file
graph.render("resnet50_graph")


In [47]:
# !pip install torchviz
import tensorflow as tf
import numpy as np
import torchviz
import pydot

# Load the ResNet50 model
# model = tf.keras.applications.ResNet50(weights='imagenet')

model = tf.keras.applications.vgg16.VGG16(weights=None) 
# model =  tf.keras.applications.resnet.ResNet152(weights='imagenet', include_top=False) 
# model =  tf.keras.applications.InceptionResNetV2(weights=None) 

# Convert the model summary to a PyDot graph
dot = tf.keras.utils.model_to_dot(model)

# Convert the PyDot graph to a PDF file
pdf_bytes = dot.create_pdf()
# with open('resnet50.pdf', 'wb') as f:
with open('vgg16.pdf', 'wb') as f:
# with open('InceptionResNetV2.pdf', 'wb') as f:
    f.write(pdf_bytes)
